In [1]:
import scipy.io
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout, BatchNormalization  
from tensorflow.keras.callbacks import EarlyStopping

# Ruta local
folder_path = r'C:\Users\itsal\Desktop\TELECO\TFG\myTFG\AE_Metodos\R_M3_v3'

# Listar todos los archivos .mat en la carpeta
files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

# Separar archivos en entrenamiento (18 ficheros) y prueba (6 ficheros)
train_files = files[:18]
test_files = files[18:]

# Función para cargar archivos .mat y extraer los datos
def load_files(file_list):
    dataframes = []
    for file in file_list:
        file_path = os.path.join(folder_path, file)
        mat_data = scipy.io.loadmat(file_path)
        df = pd.DataFrame({
            'alpha_3': mat_data['alpha_3'].flatten(),
            'beta_3': mat_data['beta_3'].flatten(),
            'delta_3': mat_data['delta_3'].flatten(),
            'gamma_3': mat_data['gamma_3'].flatten(),
            'es_lunes': mat_data['es_lunes'].flatten(),
            'es_martes': mat_data['es_martes'].flatten(),
            'es_miercoles': mat_data['es_miercoles'].flatten(),
            'es_jueves': mat_data['es_jueves'].flatten(),
            'es_viernes': mat_data['es_viernes'].flatten(),
            'es_sabado': mat_data['es_sabado'].flatten(),
            'es_domingo': mat_data['es_domingo'].flatten(),
            'horas_decimal': mat_data['horas_decimal'].flatten(),
            'energia': mat_data['energia'].flatten(),
            'porcentajes': mat_data['porcentajes'].flatten()  # Asumiendo que porcentajes es la variable objetivo
        })
        dataframes.append(df)
    
    # Concatenar todos los DataFrames en uno solo para la lista de archivos
    return pd.concat(dataframes, ignore_index=True)

In [2]:
# Cargar los archivos de entrenamiento
train_data = load_files(train_files)
X_train = train_data.drop(columns=['porcentajes'])  # Características
y_train = train_data['porcentajes']  # Etiqueta

# Cargar los archivos de prueba
test_data = load_files(test_files)
X_test = test_data.drop(columns=['porcentajes'])  # Características
y_test = test_data['porcentajes']  # Etiqueta

# Convertir los DataFrames a NumPy arrays para asegurar compatibilidad con TensorFlow
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

# Escalar las características (X) usando StandardScaler
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Escalar la salida (y) usando MinMaxScaler para que esté en el rango [0, 1]
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))  # Eliminar .values
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))  # Eliminar .values

# Reshape para LSTM: [samples, timesteps, features]
X_train_scaled = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_scaled = np.reshape(X_test_scaled, (X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))


# Construir el modelo de la red neuronal
model = Sequential()

# Capa de entrada
model.add(Input(shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))

# Primera capa LSTM
model.add(LSTM(100, return_sequences=True, activation='tanh'))  # Primera capa LSTM con más neuronas
model.add(Dropout(0.3))  # Dropout para evitar sobreajuste
model.add(BatchNormalization())  # Normalización por lotes

# Segunda capa LSTM
model.add(LSTM(50, return_sequences=True, activation='tanh'))  # Segunda capa LSTM
model.add(Dropout(0.2))

# Tercera capa LSTM
model.add(LSTM(25, return_sequences=False, activation='tanh'))  # Tercera capa LSTM para capturar patrones más complejos
model.add(Dropout(0.2))

# Capa densa final con activación 'sigmoid' para salida entre 0 y 1
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# EarlyStopping para detener el entrenamiento si no hay mejoras
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo
model.fit(X_train_scaled, y_train_scaled, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test_scaled), callbacks=[early_stopping])

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test_scaled)

# Invertir la escala de las predicciones
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2 Score: {r2}')


Epoch 1/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1345s 5ms/step - loss: 0.0071 - val_loss: 0.0234
Epoch 2/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1474s 6ms/step - loss: 0.0053 - val_loss: 0.0175
Epoch 3/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 2217s 9ms/step - loss: 0.0050 - val_loss: 0.0137
Epoch 4/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1428s 6ms/step - loss: 0.0048 - val_loss: 0.0135
Epoch 5/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1474s 6ms/step - loss: 0.0047 - val_loss: 0.0128
Epoch 6/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1489s 6ms/step - loss: 0.0046 - val_loss: 0.0123
Epoch 7/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1471s 6ms/step - loss: 0.0046 - val_loss: 0.0130
Epoch 8/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1520s 6ms/step - loss: 0.0045 - val_loss: 0.0123
Epoch 9/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1665s 7ms/step - loss: 0.0045 - val_loss: 0.0117
Epoch 10/10
249701/249701 ━━━━━━━━━━━━━━━━━━━━ 1632s 7ms/step - loss: 0.0044 - val_loss: 0.0115
64693/64693 ━━━━━━━━━━━━━━━━━━━━ 181s 3ms/step
MS

In [6]:
import scipy.io as sio
# Guardar las predicciones y los valores originales en un archivo .mat
data_to_save = {
    'y_pred': y_pred.flatten(),  # Asegurarse de que sea un array unidimensional
    'y_test': y_test.flatten()   # Asegurarse de que sea un array unidimensional
}

# Guardar el archivo .mat
sio.savemat('salida_red_epoch25_batchsize32_LSTM_v3_tanh.mat', data_to_save)